In [1]:
import numpy as np
import statsmodels.api as sm

from gmm import GELEstimator
np.random.seed(94305)

# Generalized Empirical Likelihood Estimation

We want to estimate a parameter implicitly defined as a solution to a moment condition $g(Z; \theta) = 0$. 

GMM proceeds by minimizing the quadratic form $Q(\theta) = g(Z; \theta)'Wg(Z; \theta)$, where $W$ is a positive definite weighting matrix. The choice of the weighting matrix is crucial for the efficiency of the estimator.

GEL sidesteps the choice of the weighting matrix by minimizing the empirical likelihood function. The empirical likelihood function is defined as the maximum of the likelihood function subject to the moment condition.

The (log) empirical likelihood function is defined as:

$$
L(\theta) = \max_{p \in \mathcal{P}} \sum_{i=1}^n \log p_i
$$

subject to the moment condition $$p_i g(Z; \theta) = 0$$
and adding up $\sum_{i=1}^n p_i = 1$.

where $\mathcal{P}$ is the set of all probability distributions on $\{1, \ldots, n\}$. At a first glance, this problem seems harder, since we need to solve for a weight vector $p$, which is typically of much higher dimension than the parameter $\theta$.


[Imbens, Spady, and Johnson (1998)](https://scholar.harvard.edu/sites/scholar.harvard.edu/files/imbens/files/information_theoretic_approaches_to_inference_in_moment_condition_models.pdf) show that minimizing the KLIC instead of the log empirical likelihood produces better behavior under `mild' misspecification. The corresponding Exponential Tilting estimator is defined as solving

$$
\max_{\theta, p_i} - \sum_{i=1}^n p_i \log p_i
$$

subject to the moment condition $p_i g(Z; \theta) = 0$ and $\sum_{i=1}^n p_i = 1$. 

The estimating equations are difficult to solve directly, so we instead use the saddlepoint representation

$$
\max_{\theta} \min_{t} \sum_{i=1}^n \exp(t' g(Z_i; \theta))
$$

that concentrates out the weight vector.
The inner function is strictly convex in $t$, and can be solved quickly. The outer function iterates over candidate values of $\theta$. We implement this approach in `GELEstimator`.


## Linear Problems

### estimating a mean

In [2]:
n = 1000
X = np.random.normal(np.array([1, 2]), size=(n, 2))
np.c_[X.mean(axis=0), np.sqrt(n/(n-1) * X.var(axis=0)/n)]

array([[1.01999056, 0.03015686],
       [2.04451165, 0.03196784]])

In [3]:
moment_cond_mean = lambda D, theta: D - theta

gelmod = GELEstimator(m = moment_cond_mean)
gelmod.fit(X, np.zeros(2))
gelmod.summary()

/home/alal/Desktop/gmm/gmm/gel.py:8: RuntimeWarning: overflow encountered in exp
  return 1 - np.exp(v)


array([[1.01999208, 0.03015686],
       [2.04451432, 0.03196784]])

Analytic and GEL estimates identical.


### OLS

In [4]:
def dgp(n=100_000, beta=np.array([-0.5, 1.2]), rho=0.7, pi=np.array([0.5, -0.1])):
    ε = np.random.normal(0, 1, n)
    z = np.random.normal(0, 1, n * pi.shape[0]).reshape(n, pi.shape[0])
    # Generate endogenous x, influenced by the instrument
    x = z @ pi + ε * rho + np.random.normal(0, 1, n)
    X = np.c_[np.ones(n), x]
    # heteroskedasticity
    y = X @ beta + ε + np.random.normal(0, 1 + 0.5 * (X[:, 1] > 0), n)
    return y, X, z

In [5]:
y, X, z = dgp(rho = 0)
print(sm.OLS(y, X).fit(cov_type = "HC2").summary().tables[1])

                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4973      0.005    -97.051      0.000      -0.507      -0.487
x1             1.2029      0.005    264.132      0.000       1.194       1.212


__Estimation via EL/ET__

In [6]:
def moment_condition_ols(D, θ):
    y, X = D[:, 0], D[:, 1:]
    r = y - X @ θ
    return X * r[:, None]

D = np.c_[y, X]
gelmod = GELEstimator(m = moment_condition_ols)
k = X.shape[1]
gelmod.fit(D, np.zeros(k))
gelmod.summary()

/home/alal/Desktop/gmm/gmm/gel.py:8: RuntimeWarning: overflow encountered in exp
  return 1 - np.exp(v)
/home/alal/miniforge3/envs/py311/lib/python3.11/site-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


array([[-0.49733268,  0.00512474],
       [ 1.20289956,  0.0057488 ]])

### IV

In [7]:
y, X, Z = dgp(rho = 1)
print(sm.OLS(y, X).fit(cov_type = "HC2").summary().tables[1])

                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4993      0.005   -106.700      0.000      -0.508      -0.490
x1             1.6436      0.003    528.685      0.000       1.638       1.650


Biased. 

In [8]:
# 2SLS via control function
D = X[:, 1]
Dhat = sm.OLS(D, Z).fit().predict()
tslsreg = sm.OLS(y, np.c_[sm.add_constant(D), D - Dhat]).fit(cov_type="HC1")
tslsreg.params[1], tslsreg.bse[1]

(1.2032850786188123, 0.008980883046787019)

unbiased.

In [9]:
def moment_condition_iv(D, θ):
    y, X, Z = D[:, 0], D[:, 1:3], D[:, 3:]
    r = y - X @ θ
    return Z * r[:, None]


D = np.c_[y, X, Z]
k = X.shape[1]
gelmod = GELEstimator(m=moment_condition_iv)
gelmod.fit(D, np.zeros(k))
gelmod.summary()

/home/alal/Desktop/gmm/gmm/gel.py:8: RuntimeWarning: overflow encountered in exp
  return 1 - np.exp(v)
/home/alal/miniforge3/envs/py311/lib/python3.11/site-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/home/alal/miniforge3/envs/py311/lib/python3.11/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


array([[-0.14549332,  0.00521293],
       [ 1.20631701,  0.00526045]])

Endogenous coefficient is correct

## Other Problems 

### Lin Regression

$$
Y_i = \alpha + \tau Z_i + X_i' \beta + Z_i \tilde{X}_i' \gamma + \epsilon_i
$$

where $\tilde{X}_i = X_i - \bar{X}$ is the centered version of $X_i$. 

In [10]:
n = 500
X = np.random.normal(0, 1, n * 2).reshape(n, 2)
Y0 = X[:, 0] + X[:, 0] ** 2 + np.random.uniform(-0.5, 0.5, n)
Y1 = X[:, 1] + X[:, 1] ** 2 + np.random.uniform(-1, 1, n)
Z = np.random.binomial(1, 0.6, n)
Y = Y0 * (1 - Z) + Y1 * Z
D = np.c_[Y, Z, X]

In [11]:
sm.OLS(
    Y, np.c_[sm.add_constant(Z), X, Z[:, None] * (X - X.mean(axis=0))]
).fit().summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,1.0189,0.117,8.721,0.000,0.789,1.248
x1,0.1907,0.149,1.281,0.201,-0.102,0.483
x2,0.9860,0.118,8.365,0.000,0.754,1.218
x3,0.1129,0.115,0.977,0.329,-0.114,0.340
x4,-0.8281,0.150,-5.519,0.000,-1.123,-0.533
x5,1.4133,0.142,9.927,0.000,1.134,1.693


The point estimates are consistent, but we do not propagate forward the uncertainty from estimating the sample mean $\bar{X}$. To do this, we could stack the moment conditions

$$
\begin{bmatrix}
X - \mu \\
[X, X - \mu] (y - [X, X - \mu] \beta)
\end{bmatrix}
$$

In [12]:
def moment_condition_lin(D, θ):
    Y, Z, X = D[:, 0], D[:, 1], D[:, 2:]
    n, p = X.shape
    mu, beta = θ[:p], θ[p:]
    Xcent = X - mu
    XX = np.c_[np.ones(n), Z, X, Z[:, None] * Xcent]
    r = XX * (Y - XX @ beta)[:, None]
    return np.c_[Xcent, r]

In [14]:
print(theta_init := np.r_[X.mean(axis=0), np.random.rand(6)])

[0.02681823 0.0657933  0.94022998 0.59071843 0.16238761 0.18472579
 0.70219239 0.66758587]


In [18]:
gelmod = GELEstimator(m=moment_condition_lin)
gelmod.fit(D, theta_init)
gelmod.summary()

/home/alal/Desktop/gmm/gmm/gel.py:8: RuntimeWarning: overflow encountered in exp
  return 1 - np.exp(v)
/home/alal/miniforge3/envs/py311/lib/python3.11/site-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan]])

!TODO look into why this fails to converge.